In [4]:
import pandas as pd

In [20]:
def rgb_string_to_tuple(rgb_string):
    """ Convert an RGB string 'rgb(x, y, z)' to a normalized RGB tuple. """
    rgb = rgb_string.strip('rgb()').split(',')
    return tuple(int(num) / 255 for num in rgb)

def get_colors(style, inverted):
    """ Return appropriate colors based on style and inversion. """
    color_config = {
        'col_scale': ['rgb(214, 95, 95)', 'rgb(95, 186, 125)'],  # Red to Green
        'bar': ['rgb(214, 95, 95)', 'rgb(95, 186, 125)']  # Red for negative, Green for positive
    }
    
    if style not in color_config:
        raise ValueError(f"Style {style} is not supported. Available styles: {list(color_config.keys())}")
    
    colors = color_config[style]
    return colors[::-1] if inverted else colors

def initialize_styler(df):
    """ Initialize DataFrame styler with common properties. """
    return df.style.set_properties(
        **{'background-color': 'white', 'text-align': 'center', 'border': '1px solid black'}
    ).set_table_styles(
        [{'selector': 'th', 'props': [('text-align', 'center')]}]
    )
    
def apply_bar_style(df, columns, colors):
    """ Apply bar style to columns with specified RGB colors. """
    styler = initialize_styler(df)
    for column in columns:
        styler = styler.bar(subset=[column], align='mid', color=colors)
    return styler

def apply_gradient_style(df, column, colors, q_low, q_high):
    """ Apply gradient style to a column based on quantiles with given RGB colors. """
    from matplotlib.colors import LinearSegmentedColormap

    # Convert RGB string colors to normalized RGB tuples
    colors = [rgb_string_to_tuple(color) for color in colors]

    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
    quantile_low = df[column].quantile(q_low / 100.0)
    quantile_high = df[column].quantile(q_high / 100.0)
    styler = initialize_styler(df)
    return styler.background_gradient(subset=[column], cmap=cmap, vmin=quantile_low, vmax=quantile_high, axis=0)

def apply_custom_style(df, columns, style, params=None, inverted=False):
    if not isinstance(columns, list):
        columns = [columns]  # Ensure columns is a list for uniform processing

    colors = get_colors(style, inverted)

    if style == 'col_scale':
        if not params or len(params) != 2:
            raise ValueError("Params must be a tuple with two percentage values for 'col_scale'")
        # Initialize the styler from the DataFrame
        styler = initialize_styler(df)
        q_low, q_high = params
        for column in columns:
            # Apply gradient style and keep chaining the styling
            styler = apply_gradient_style(df, column, colors, q_low, q_high)
        return styler  # Return the fully styled DataFrame

    elif style == 'bar':
        return apply_bar_style(df, columns, colors)

    else:
        raise ValueError("Unsupported style option. Choose 'col_scale', 'bar', or 'icon'")

In [25]:
import pandas as pd

# Example DataFrame
data = {
    'Product': ['Laptop', 'Printer', 'Tablet', 'Desk', 'Chair'],
    'Sales': [1000, 300, 450, 150, 700],
    'Profit': [200, -60, 90, -30, 140]
}
df = pd.DataFrame(data)

In [26]:
# Apply bar style
styled_df_bar = apply_custom_style(df, ['Profit'], 'bar', inverted=True)
# Display styled DataFrames in a Jupyter Notebook:
styled_df_bar

,Product,Sales,Profit
0,Laptop,1000,200
1,Printer,300,-60
2,Tablet,450,90
3,Desk,150,-30
4,Chair,700,140


In [24]:
# Apply color scale style
styled_df_col_scale = apply_custom_style(df, ['Sales'], 'col_scale', params=(10, 90), inverted=False)
styled_df_col_scale

,Product,Sales,Profit
0,Laptop,1000,200
1,Printer,300,-60
2,Tablet,450,90
3,Desk,150,-30
4,Chair,700,140
